[View in Colaboratory](https://colab.research.google.com/github/BurgerAutomata/PrincetonAlgorithms/blob/master/GraphMinCut.ipynb)

# Programming Assignment 4
<br>

find the minimum cut of a graph.
<br>
Your task is to code up and run the randomized contraction algorithm for the min cut problem and use it on the above graph to compute the min cut. (HINT: Note that you'll have to figure out an implementation of edge contractions. Initially, you might want to do this naively, creating a new graph from the old every time there's an edge contraction. But you should also think about more efficient implementations.) (WARNING: As per the video lectures, please make sure to run the algorithm many times with different random seeds, and remember the smallest cut that you ever find.) Write your numeric answer in the space provided. So e.g., if your answer is 5, just type 5 in the space provided.
<br>
Types yet unimplemented
<br>
+https://teacode.wordpress.com/2013/07/19/karger-min-cut-problem/

+https://datarecipe.wordpress.com/2015/07/23/python-practice-kargers-contraction-min-cut-algorithm/

+https://stackoverflow.com/questions/49810171/karger-min-cut-for-coursera-algorithms-course-python-3-6


In [2]:
import numpy as np
import sys
import pandas as pd
import random as rnd
import copy

In [33]:
#neat import from local files
#local_file=pd.read_table('kargerMinCut.txt',header=None,sep="\t",lineterminator='\r')
#From the forum, much more efficient because this is a "frayed" array
input_file = 'kargerMinCut.txt'
with open(input_file, 'r') as data:
    line = data.read().strip().split("\n")

graph_dict = {}
for element in line:
    line_list = list(map(int, element.strip().split("\t")))
    graph_dict[str(line_list[0])] = line_list[1:]

In [36]:
#Points in the graph
type(list(graph_dict.keys())[1])
#graph_dict['1']

str

In [6]:
#Test cases
test_Dict={'A':{'1': [2, 3, 4, 7],'2': [1, 3, 4],'3': [1,2,4],'4': [1,2,3,5],'5': [4,6,7,8],'6': [5,7,8],'7': [1,5,6,8],'8': [5,6,7]},
           'B':{'1': [4,2,7,3],'2': [4,1,3],'3': [1,2,4],'4': [5,1,2,3],'5': [8,7,6,4],'6': [8,5,7],'7': [6,8,5,1],'8': [7,6,5]},
            'C':{'1': [2,3,4],'2': [1,3,4],'3': [1,2,4],'4': [1,2,3,5],'5': [4,6,7,8],'6': [5,7,8],'7': [5,6,8],'8': [5,6,7]},
          'D':{'1': [3,4,2],'2': [1,4,3],'3': [1,2,4],'4': [5,3,2,1],'5': [4,8,6,7],'6': [8,7,5],'7': [5,8,6],'8': [5,7,6]},
          'E':{'1': [19,15,36,23,18,39],'2': [36,23,4,18,26,9],'3': [35,6,16,11],'4': [23,2,18,24],'5': [14,8,29,21],'6': [34,35,3,16],
                '7': [30,33,38,28],'8': [12,14,5,29,31],'9': [39,13,20,10,17,2],'10': [9,20,12,14,29],'11': [3,16,30,33,26],
               '12': [20,10,14,8],'13': [24,39,9,20],'14': [10,12,8,5],'15': [26,19,1,36],'16': [6,3,11,30,17,35,32],
                '17': [38,28,32,40,9,16],'18': [2,4,24,39,1],'19': [27,26,15,1],'20': [13,9,10,12],'21': [5,29,25,37],
                '22': [32,40,34,35],'23': [1,36,2,4],'24': [4,18,39,13],'25': [29,21,37,31],'26': [31,27,19,15,11,2],
                '27': [37,31,26,19,29],'28': [7,38,17,32],'29': [8,5,21,25,10,27],'30': [16,11,33,7,37],'31': [25,37,27,26,8],
                '32': [28,17,40,22,16],'33': [11,30,7,38],'34': [40,22,35,6],'35': [22,34,6,3,16],'36': [15,1,23,2],
                '37': [21,25,31,27,30],'38': [33,7,28,17,40],'39': [18,24,13,9,1],'40': [17,32,22,34,38]},
          'F':{'1': [2,3,4,5],'2': [3,4,1],'3': [4,1,2],'4': [1,2,3,8],'5': [1,6,7,8],'6': [7,8,5],'7': [8,5,6],'8': [ 4,6,5,7]}}


ans_Dict={'A':2,'B':2,'C':1,'D':1,'E':3,'F':2}


In [7]:
len(test_Dict['A'])
#verify that len actually sees the number of keys

#try out update method
txt1=test_Dict['A'].copy()
print(txt1['1'])
print(txt1['2'])
indChos=rnd.choice(list(txt1.keys()))
print(indChos)
print(type(indChos))
print(txt1[indChos])
chosenVertex=rnd.choice(txt1[indChos])
print(chosenVertex)
print(type(chosenVertex))
print(txt1['1'][2])

[2, 3, 4, 7]
[1, 3, 4]
6
<class 'str'>
[5, 7, 8]
5
<class 'int'>
4


expected result: 2

cuts are [(1,7), (4,5)]

(randomly permuting the adjacency list, should get same result):


expected result: 2

cuts are [(1,7), (4,5)]



expected result: 1

cut is [(4,5)]

(randomly permuting the adjacency list, should get same result):


expected result: 1

cut is [(4,5)]


expected result: 3


mincut=2

## Counter Implementation
This link provides a particularly interesting case, as it does not use classes<br>
However it seems maybe it works in Python 2
<br>
https://github.com/reetawwsum/Karger-Min-Cut/blob/master/karger.py

In [8]:
from random import choice
from collections import Counter
from copy import deepcopy

g = {}

def kargerMinCut(g):
	while len(g) > 2:
		#Selecting a random vertex
		u = choice(list(g.keys()))
		gu = g[u]
		#Selecting most comman vertex among the previously chosen random vertex
		v = gu.most_common(1)[0][0]
		gv = g[v]
		#Deleting second vertex from the graph
		del g[v]
		#Deleting self loop
		del gv[u]
		del gu[v]
		#Merging second vertex into first vertex
		gu.update(gv)
		for w in gv:
			gw = g[w]
			gw[u] += gw[v]
			del gw[v]
	return g.itervalues().next().most_common(1)[0][1]

with open('kargerMinCut.txt', 'r') as graphInput:
	for line in graphInput:
		ints = [int(x) for x in line.split()]
		g[ints[0]] = Counter(ints[1:])

cuts = [kargerMinCut(deepcopy(g)) for x in range(5)]

AttributeError: 'dict' object has no attribute 'itervalues'

## Data Recipe implementation
Another one based on classes. This one seems to be quite slow.

In [ ]:
import random
 
class Node:
    def __init__(self, id):
        self.id = id
        self.edges = {}
    def add(self, id, num):
        if id != self.id:
            self.edges[id] = self.edges.get(id, 0) + num 
    def remove(self, id):
        try:
            del self.edges[id]
        except KeyError:
            pass
    def merge(self, id1, id2):
        if id2 in self.edges:
            self.edges[id1] = self.edges.get(id1, 0) + self.edges[id2]            
            self.remove(id2)
    def size(self):
        return sum(self.edges.values())
    def copy(self):
        node = Node(self.id)
        node.edges = dict(self.edges)
        return node
    def __equals__(self, other):
        return self.id == other.id
    def __str__(self):
        return str({self.id : self.edges})
 
 
class graph:
    def __init__(self, nodes): 
        self.nodes = {node.id : node.copy() for node in nodes}
    def combine(self, id1, id2):
        if id1 not in self.nodes or id2 not in self.nodes[id1].edges:
            return
        for id in self.nodes[id2].edges:
            self.nodes[id1].add(id, self.nodes[id2].edges[id])
        try:
            del self.nodes[id2]
            del self.nodes[id1].edges[id2]
        except KeyError:
            pass
        for id in self.nodes:
            self.nodes[id].merge(id1, id2)
    def randomCombine(self):
        node1 = random.choice(list(self.nodes.values()))
        id1 = node1.id
        id2 = random.choice(list(node1.edges.keys()))
        self.combine(id1, id2)
    def size(self):
        return len(self.nodes)
    def __str__(self):
        return '\n'.join([str(node) for node in self.nodes.values()])
 
 
f = open('./kargerMinCut.txt', 'r')
lines = f.readlines()
n = len(lines)
nodes = []
for line in lines:
    ids = [int(id) for id in line.split()]
    node = Node(ids[0])
    for id in ids[1:]:
        node.add(id,1)
    nodes.append(node)
 
rawSet = graph(nodes)
#print(rawSet)
 
cuts = []
for i in range(n**2):
    rawSet = graph(nodes)
    while rawSet.size() > 2:
        rawSet.randomCombine()
    id1 = list(rawSet.nodes.keys())[0]
    id2 = list(rawSet.nodes.keys())[1]
    size1 = rawSet.nodes[id1].size()
    size2 = rawSet.nodes[id2].size()
    assert size1 == size2, "Node size not equal: %d and %d" % (size1, size2)
    cuts.append(size1)
 
min(cuts)

## My Implementation
<br> instead is our implementation

In [52]:
#algorithm
def minCut(graph):
    listCuts=[]
    #Internal deep copy for safety
    graphInt=copy.deepcopy(graph)
    
    #reduce graph in place, as we do a copy anyway
    #This loop stops when there is only 2 nodes left
    while len(graphInt)>2:
        #inner loop where cut happens
        #The following is an attempt to represent picking any edge at random.
        #The Edge will be (cutSelect1, cutSelect2)
        #select node. This is a STRING
        cutSelect1=rnd.choice(list(graphInt.keys()))
        #print('First Node selected ', cutSelect1)
        #select node connected to previous. Remember this is an INT
        cutSelect2=rnd.choice(graphInt[cutSelect1])
        #print('Second Node selected ', cutSelect2)

        #collapse them together in a temporary node
        tempNode=graphInt[cutSelect1]+graphInt[str(cutSelect2)]
        #print('temporary collapse ', tempNode)
        #Remove references to itself, so to first and second node
        tempNode=[i for i in tempNode if (i!=int(cutSelect1) and (i!=cutSelect2))]
        #print('cleaned collapse ', tempNode)
        #remove second vertex from graph
        #print('Check the changes A',graphInt)
        del graphInt[str(cutSelect2)]
        #Change first vertex to new list
        graphInt[cutSelect1]=tempNode.copy()
        #print('Check the changes B',graphInt)

        #Change reference in all other nodes?
        for j in list(graphInt.keys()):
            #j should be a string
            #skip at least this node
            if j!=cutSelect1:
                #print('Working on connections of node ',j,' ',graphInt[j])
                for i,k in enumerate(graphInt[j]):
                    #i should be int INDEX, k the actual list element
                    
                    if k ==cutSelect2:
                        graphInt[j][i]=int(cutSelect1)
        
                #print('Modified connections ', graphInt[j] )
        #remove self loops
        #print('Check the changes C',graphInt)

    #Should count the amounts of links left
    #print(graphInt)
    listCuts=graphInt[list(graphInt.keys())[0]]
    #print(len(listCuts))
    return len(listCuts)

def repeatMinCut(graph):
    #execute the same loop many times
    listMins=[]
    for i in range(len(graph)*5):
        curMin=minCut(graph)
        listMins.append(curMin)
    return min(listMins)
        

In [39]:
#Test loop
tests=['A','B','C','D','E','F']
minCut(test_Dict['C'])
'''for name in tests:
    loc_test=test_Dict[name].copy()
    loc_answ=ans_Dict[name]
    listCuts,minCuts=minCut(loc_test)
    #Verify results
    print(minCuts)'''
    
    

'for name in tests:\n    loc_test=test_Dict[name].copy()\n    loc_answ=ans_Dict[name]\n    listCuts,minCuts=minCut(loc_test)\n    #Verify results\n    print(minCuts)'

In [53]:
repeatMinCut(test_Dict['E'])

3

In [54]:
repeatMinCut(graph_dict)

17